## Import usefull libs

In [ ]:
!pip install setuptools-rust

In [ ]:
!pip install transformers==2.5.1

In [ ]:
!pip install --upgrade diffusers

In [ ]:
!pip install img2dataset

In [ ]:
!pip install wandb

In [ ]:
import torch
import time
from datetime import datetime
from tqdm import tqdm
import pandas as pd
from google.colab import drive
import os
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import numpy as np
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, KFold
from sklearn.multiclass import OneVsRestClassifier
from PIL import Image
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd
import csv
from urllib.request import urlopen
import multiprocessing
import wandb
import tarfile

## Correct dataset loading by API

In [ ]:
# Making directory for url-list files
!mkdir -p laion-high-resolution && cd laion-high-resolution
# Downloading files from url
!for i in {00000..00001}; do wget -P laion-high-resolution/ https://huggingface.co/datasets/laion/laion-high-resolution/resolve/main/part-$i-5d6701c4-b238-4c0a-84e4-fe8e9daea963-c000.snappy.parquet; done

In [ ]:
# Loginig in wandb
wandb.login()

In [ ]:
#!rm -rf /content/laion-high-resolution-output

In [ ]:
# Downloading .tar by url-list files from created directory
# Do not forget to stop running after completing the load
!img2dataset --url_list laion-high-resolution --input_format "parquet"\
         --url_col "URL" --caption_col "TEXT" --output_format webdataset\
           --output_folder laion-high-resolution-output --processes_count 16 --thread_count 64 --image_size 1024\
            --resize_only_if_bigger=True --resize_mode="keep_ratio" --skip_reencode=True \
             --save_additional_columns '["similarity","hash","punsafe","pwatermark","LANGUAGE"]' --enable_wandb True

In [ ]:
# Creating dataset directory for ready files
!mkdir -p dataset

In [ ]:
# Function encode .tar by the path
# Files will encode in /content/, do not forget to use next func
def UnboxTar(tar_path = "/content/laion-high-resolution-output/00000.tar"):
  try:
      tar = tarfile.open(tar_path)
      tar.extractall()
      tar.close()
  except Exception:
      print('Error')

In [ ]:
UnboxTar()

In [ ]:
# Move all files from /content/ to file_path directory
def ReloadAllToFrom(file_path = 'dataset', directory = '/content/'):
  !mkdir -p {file_path}
  for filename in os.listdir(directory):
    # os.path.join соединяет путь до директории и имя файла
    path = os.path.join(directory, filename)
    # os.path.isfile проверяет, является ли путь файлом (а не папкой)
    if os.path.isfile(path):
      !mv {path} {path[:path.rfind('/') + 1] + file_path + '/' + path[path.rfind('/') + 2:]}

In [ ]:
# Move image files from /content/ to file_path directory
def ReloadImgFromTo(from_path = 'dataset', to_path = 'dataset_images'):
  !mkdir -p {to_path}
  for filename in os.listdir(from_path):
    # os.path.join соединяет путь до директории и имя файла
    path = os.path.join(from_path, filename)
    # os.path.isfile проверяет, является ли путь файлом (а не папкой)

    if os.path.isfile(path):
      if path[path.find('.') + 1:] == 'jpg' or path[path.find('.') + 1:] == 'png' or path[path.find('.') + 1:] == 'jpeg':
        !mv {path} {to_path + '/' + path[path.rfind('/') + 2:]}

In [ ]:
ReloadAllToFrom()

In [ ]:
ReloadImgFromTo()

As a result we have directory (named dataset in default situation) with text, images and describes from the dataset.

## Load dataset by parsing (bad alternative)

Make data directory, go there

In [ ]:
!mkdir laion-high-resolution && cd laion-high-resolution

In [ ]:
# Function for loading 100 rows of dataset from the url
# Param filtering indicate filtering of data by languages (langs) when it is filtering = True
# Param filter_lang sorts data from all langs except filter_lang
# Param file_name sets file_name + .csv
def LoadLaionDataset(filtering = False, filter_lang = '', load_csv = False, file_name = 'dataset', url = 'https://huggingface.co/datasets/laion/laion-high-resolution/viewer/default/train'):
  # Created an URL object in url
  # Create object page
  page = requests.get(url)
  # parser-lxml = Change html to Python friendly format
  # Obtain page's information
  soup = BeautifulSoup(page.text, 'lxml')
  # Obtain information from tag <table>
  table = soup.find('table', attrs={"class": "w-full table-auto rounded-lg font-mono text-xs text-gray-900"})
  # Obtain every title of columns with tag <th>
  headers = []
  for i in table.find_all('th'):
    title = i.text
    headers.append(title[:title.find('\n')])
  # Create a dataframe
  df = pd.DataFrame(columns = headers)
  # Create a for loop to fill mydata
  for j in table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text[:i.text.find('\n')] for i in row_data]
    for i in range(len(row)):
      row[i] = row[i][row[i].find('"') + 1: row[i].rfind('"')]
    length = len(df)
    df.loc[length] = row
  # Filtering by languages
  if filtering:
    df = df[df['LANGUAGE'] == filter_lang]
  # Loading .csv
  if load_csv:
    df.to_csv ('/content/laion-high-resolution/' + str(file_name) + '.csv', index= False )
  return df

In [ ]:
df = LoadLaionDataset(load_csv = True)

In [ ]:
df.head()

Upgrade of dataset loading function

In [ ]:
# Param frames_count sets count of loading url pages
# Other params are same eith previous function
def LoadData(frames_count = 1, filtering = False, filter_lang = '', load_csv = False, file_name = 'dataset', based_url = 'https://huggingface.co/datasets/laion/laion-high-resolution/viewer/default/train'):
  # DataFrame list
  df_list = list()
  # Updating df_list by loading .csv by url
  for i in tqdm(range(frames_count)):
    df_list.append(LoadLaionDataset(filtering, filter_lang, load_csv = False, url = based_url + '?p=' + str(i)))
  # concating df_list to one big DataFrame
  result_df = pd.concat(df_list)
  X = result_df[['TEXT', 'LANGUAGE']].to_numpy()
  y = result_df['URL'].to_numpy()
  return result_df, list([X, y])

In [ ]:
df, dataset = LoadData(frames_count = 1)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
dataset[0][0]

## KFold sampling on train-test split



In [ ]:
# This func makes KFold from Sklearn more convinient to out task
def KFoldSampling(dataset, nsplits):
  # X, y split from dataset
  X, y = dataset[0], dataset[1]
  kf = KFold(n_splits = nsplits)
  kf.get_n_splits()
  # list of indicies of KFold
  result = list()
  for i, (train_index, test_index) in tqdm(enumerate(kf.split(X))):
    # Appending train-test indicies ro result list
    result.append(np.array([train_index, test_index]))
  return np.array(result)

In [ ]:
KFoldSampling(dataset, nsplits=2)

## Dataset transform from url to image

In [ ]:
# Function for loading image
def LoadImageMulti(url, queue):
  response = requests.get(url)
  img = Image.open(BytesIO(response.content))
  queue.put(img)

# Safe loading of image with time limit

def LineStop(target, arg, time_limit = 10):
  queue = multiprocessing.Queue()
  p = multiprocessing.Process(target=target, args=(arg, queue))
  p.start()
  p.join(time_limit)
  if p.is_alive():
    print('ABORT')
    # Terminate
    p.terminate()
    return
  return queue.get()

LineStop(LoadImageMulti, 'https://mmedia.ozone.ru/multimedia/1018085734.JPG')

In [ ]:
def DatasetTransform(dataset, image_dir = '', time_limit = 10, all_time_limit = 1000):
  # Making directory for dataset
  if image_dir:
    if image_dir[len(image_dir) - 1] == '/':
      image_dir = image_dir[:len(image_dir) - 1]
    os.environ["DIRECTORY_NAME"] = image_dir
    !mkdir -p ${DIRECTORY_NAME}
  # Making X, y from dataset
  X = dataset[0]
  y = dataset[1]
  # Making result lists
  result_y = list()
  result_X = list()
  # Start time
  start_time = datetime.now()

  for i, url in enumerate(tqdm(y)):
    # All exceptions are ignored
    try:
      # Load image
      response = requests.get(url)
      img = Image.open(BytesIO(response.content))
      if image_dir:
        print(image_dir + '/image_' + str(i) + '.jpg')
        img.save(image_dir + '/image_' + str(i) + '.jpg')
    except Exception:
      continue
    # Appending numpy image
    result_y.append(np.array(img))
    result_X.append(X[i])
    # Checking time break
    if (datetime.now() - start_time).seconds > all_time_limit:
      print('Time break')
      return list([np.array(result_X), np.array(result_y)])
  return list([np.array(result_X), np.array(result_y)])

In [ ]:
DatasetTransform(dataset, image_dir='/content/data_images', all_time_limit = 30)

## SDXL PART

## Loading packages

Downloading & Importing smth

In [ ]:
!pip install --upgrade safetensors

In [ ]:
!pip install transformers

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install --upgrade diffusers

In [ ]:
from huggingface_hub import snapshot_download
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline

## Refactoring directory by matching images

In [ ]:
def MatchImagesDir(directory = '/content/data_images', target_dir = '/content/matched_images'):
  list_img = list()
  for filename in os.listdir(directory):
    # os.path.join соединяет путь до директории и имя файла
    path = os.path.join(directory, filename)
    # os.path.isfile проверяет, является ли путь файлом (а не папкой)

    if os.path.isfile(path):
      if path[path.find('.') + 1:] == 'jpg' or path[path.find('.') + 1:] == 'png' or path[path.find('.') + 1:] == 'jpeg':
        list_img.append(path)
  for i in range(len(list_img) - 1):
    # Read images
    img_left = Image.open(list_img[i])
    img_right = Image.open(list_img[i + 1])
    # Reading sizes
    width_left, height_left = img_left.size
    width_right, height_right = img_right.size
    # Counting new sizes
    result_width = width_left + width_right
    result_height = max(height_left, height_right)
    # Making result image
    result = Image.new('RGB', (result_width, result_height))
    # Refactoring result image
    result.paste(im=img_left, box=(0, 0))
    result.paste(im=img_right, box=(width_left, 0))
    # Save result image
    result.save(target_dir + '/matched' + str(i) + '.jpg')

In [ ]:
MatchImagesDir()

## SDXL with Finetuning

Example pictures dataset

In [ ]:
local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

In [ ]:
#@title 🤗 AutoTrain DreamBooth
#@markdown In order to use this colab
#@markdown - upload images to a folder named `images/`
#@markdown - choose a project name if you wish
#@markdown - change model if you wish, you can also select sd2/2.1 or sd1.5
#@markdown - update prompt and remember it. choose keywords that don't usually appear in dictionaries
#@markdown - add huggingface information (token and repo_id) if you wish to push trained model to huggingface hub
#@markdown - update hyperparameters if you wish
#@markdown - click `Runtime > Run all` or run each cell individually

!pip install -U autotrain-advanced > install_logs.txt
!autotrain setup > setup_logs.txt

In [ ]:
#@markdown ---
#@markdown #### Project Config
project_name = 'fine_tuned_model' # @param {type:"string"}
dataset_dir = '/content/matched_images' # @param {type: "string"}
model_name = 'runwayml/stable-diffusion-v1-5' # @param ["stabilityai/stable-diffusion-xl-base-1.0", "runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-2-1", "stabilityai/stable-diffusion-2-1-base"]
prompt = 'combination of prompts' # @param {type: "string"}


#@markdown ---
#@markdown #### Push to Hub?
#@markdown Use these only if you want to push your trained model to a private repo in your Hugging Face Account
#@markdown If you dont use these, the model will be saved in Google Colab and you are required to download it manually.
#@markdown Please enter your Hugging Face write token. The trained model will be saved to your Hugging Face account.
#@markdown You can find your token here: https://huggingface.co/settings/tokens
push_to_hub = False # @param ["False", "True"] {type:"raw"}
hf_token = "hf_XXX" #@param {type:"string"}
repo_id = "username/repo_name" #@param {type:"string"}

#@markdown ---
#@markdown #### Hyperparameters
learning_rate = 1e-3 # @param {type:"number"}
num_steps = 20 #@param {type:"number"}
batch_size = 1 # @param {type:"slider", min:1, max:32, step:1}
gradient_accumulation = 4 # @param {type:"slider", min:1, max:32, step:1}
resolution = 1024 # @param {type:"slider", min:128, max:1024, step:128}
use_8bit_adam = True # @param ["False", "True"] {type:"raw"}
use_xformers = True # @param ["False", "True"] {type:"raw"}
use_fp16 = True # @param ["False", "True"] {type:"raw"}
train_text_encoder = False # @param ["False", "True"] {type:"raw"}
gradient_checkpointing = True # @param ["False", "True"] {type:"raw"}

os.environ["PROJECT_NAME"] = project_name
os.environ["DATASET_DIR"] = dataset_dir
os.environ["MODEL_NAME"] = model_name
os.environ["PROMPT"] = prompt
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
os.environ["REPO_ID"] = repo_id
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)
os.environ["USE_8BIT_ADAM"] = str(use_8bit_adam)
os.environ["USE_XFORMERS"] = str(use_xformers)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["TRAIN_TEXT_ENCODER"] = str(train_text_encoder)
os.environ["GRADIENT_CHECKPOINTING"] = str(gradient_checkpointing)

In [ ]:
!autotrain dreambooth \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--image-path ${DATASET_DIR} \
--prompt "${PROMPT}" \
--resolution ${RESOLUTION} \
--batch-size ${BATCH_SIZE} \
--num-steps ${NUM_STEPS} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--lr ${LEARNING_RATE} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_XFORMERS" == "True" ]] && echo "--xformers" ) \
$( [[ "$TRAIN_TEXT_ENCODER" == "True" ]] && echo "--train-text-encoder" ) \
$( [[ "$USE_8BIT_ADAM" == "True" ]] && echo "--use-8bit-adam" ) \
$( [[ "$GRADIENT_CHECKPOINTING" == "True" ]] && echo "--gradient-checkpointing" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}" )

In [ ]:
def ImageFromPrompt(prompt, weight_name="pytorch_lora_weights.safetensors", project_name=project_name, model=model_name):
  prj_path = "/content/" + str(project_name)
  pipe = DiffusionPipeline.from_pretrained(
      model,
      torch_dtype=torch.float16,
  )
  pipe.to("cuda")
  pipe.load_lora_weights(prj_path, weight_name=weight_name)

  refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
      "stabilityai/stable-diffusion-xl-refiner-1.0",
      torch_dtype=torch.float16,
  )
  refiner.to("cuda")

  seed = 42
  generator = torch.Generator("cuda").manual_seed(seed)
  image = pipe(prompt=prompt, generator=generator).images[0]
  image_upgrade = refiner(prompt=prompt, generator=generator, image=image).images[0]
  return image, image_upgrade

In [ ]:
img, img_upgrade = ImageFromPrompt('combination of prompts: photo of Leo Messi/photo of Cristiano Ronaldo', weight_name='model.safetensors')

In [ ]:
img

In [ ]:
img_upgrade